<a href="https://colab.research.google.com/github/starvar7169/NLPgenz/blob/main/gen-z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import torch
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report

In [ ]:
data= pd.read_csv("genz_slang.csv")
data_emoji= pd.read_csv("genz_emojis.csv")
print(data.head())
print(data_emoji.head())

   keyword                                        description
0        W                                  Shorthand for win
1        L                          Shorthand for loss/losing
2  L+ratio  Response to a comment or action on the interne...
3     Dank                  excellent or of very high quality
4   Cheugy  Derogatory term for Millennials. Used when mil...
  emoji                      Name  \
0     💀                     Skull   
1     🤡                     Clown   
2     😭             Loudly crying   
3     😅  Grinning face with sweat   
4     👍                 Thumbs up   

                                         Description  
0                       Used as a symbol of laughter  
1  someone who is acting or saying something incr...  
2  exclusively used to indicate positive feelings...  
3                        I’m stressed, but I’m fine.  
4  Used to send a sarcastic or passive-aggressive...  


In [ ]:
slang_texts = [f"{keyword}: {description}" for keyword, description in zip(data['keyword'], data['description'])]
emoji_texts = [f"{emoji} ({Name}): {Description}" for emoji, Name, Description in zip(data_emoji['emoji'], data_emoji['Name'], data_emoji['Description'])]
combined_text = slang_texts + emoji_texts
print(combined_text)

['W: Shorthand for win', 'L: Shorthand for loss/losing', 'L+ratio: Response to a comment or action on the internet that is particularly bad.', 'Dank: excellent or of very high quality', 'Cheugy: Derogatory term for Millennials. Used when millennials are perceived to be excessively attempting to be trendy or stylish.', 'TFW:  That feeling when', 'Woke:  being politically aware', 'Bop: An excellent song or album.', 'G.O.A.T:  The greatest of all time', 'Smol: Something that is small, and in most cases exceptionally adorable.', 'Fam: Fam is a shorter word for family, but don\'t be fooled– it can be used to describe your friends or the way Millennials use "bro".', 'Glow up: a makeover or transformation from bad to good.', 'Stan: Supporting something. Specifically used in some cases to the idolatry support of personalities.', 'Ghosting: common amongst the earlier talking stages of a relationship. Ghosting someone means you start ignoring them or stop texting them back.', 'Salty: Used when s

In [ ]:
dataset = Dataset.from_dict({"text": combined_text})
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Tokenize dataset with map function
tokenized_dataset = dataset.map(lambda examples: tokenizer(examples["text"], padding=True, truncation=True), batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/218 [00:00<?, ? examples/s]

In [ ]:
combined_text_series = pd.Series(combined_text)
combined_text_series.info()
print(combined_text_series)

<class 'pandas.core.series.Series'>
RangeIndex: 218 entries, 0 to 217
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
218 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB
0                                   W: Shorthand for win
1                           L: Shorthand for loss/losing
2      L+ratio: Response to a comment or action on th...
3                Dank: excellent or of very high quality
4      Cheugy: Derogatory term for Millennials. Used ...
                             ...                        
213    👩‍🦽🧎‍♀️🦽 (Woman in manual wheel chair, Woman k...
214     🎤🦆 (Microphone, Duck): Who has something to say?
215    🤰🤛 (Pregnant woman, Left-facing fist): Let it die
216                 🔭🦔 (Telescope, Hedgehog): Stargazing
217    👂🤏👂 (Ear, Pinching hand, Ear): Huh I wasn’t li...
Length: 218, dtype: object


In [ ]:
train_texts, remaining_texts = train_test_split(combined_text, test_size=0.2, random_state=42)  # Split into 80% train and 20% remaining
val_texts, test_texts = train_test_split(remaining_texts, test_size=0.5, random_state=42)  # Split remaining into 10% val and 10% test

train_dataset = Dataset.from_dict({"text": train_texts}).map(lambda x: tokenizer(x["text"], padding=True, truncation=True), batched=True)
val_dataset = Dataset.from_dict({"text": val_texts}).map(lambda x: tokenizer(x["text"], padding=True, truncation=True), batched=True)
test_dataset = Dataset.from_dict({"text": test_texts}).map(lambda x: tokenizer(x["text"], padding=True, truncation=True), batched=True)

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [ ]:
data_collator= DataCollatorForLanguageModeling(tokenizer= tokenizer, mlm= True, mlm_probability= 0.3)

In [ ]:
from transformers import AutoModelForMaskedLM, Trainer, TrainingArguments

# Load pre-trained model
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", trust_remote_code= True)

# Training arguments
training_args = TrainingArguments(
    fp16= True,
    output_dir="./genz_model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=64,
    save_steps=500,
    save_total_limit=3,
    lr_scheduler_type="cosine",
    learning_rate=5e-5,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    logging_dir="./logs",
    weight_decay = 0.02,
    metric_for_best_model= "eval_loss",
    load_best_model_at_end= True
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset= tokenized_dataset,
    eval_dataset= val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=3.773516845703125, metrics={'train_runtime': 37.7918, 'train_samples_per_second': 28.842, 'train_steps_per_second': 0.529, 'total_flos': 39223685970000.0, 'train_loss': 3.773516845703125, 'epoch': 5.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./genz_finetuned_model")
tokenizer.save_pretrained("./genz_finetuned_model")

('./genz_finetuned_model/tokenizer_config.json',
 './genz_finetuned_model/special_tokens_map.json',
 './genz_finetuned_model/vocab.txt',
 './genz_finetuned_model/added_tokens.json',
 './genz_finetuned_model/tokenizer.json')

In [ ]:
results = trainer.evaluate(eval_dataset=val_dataset)

# Extract perplexity from loss
perplexity = math.exp(results['eval_loss'])
print(f"Perplexity: {perplexity}")

Perplexity: 28.280226424459155


baseline perplexity

In [ ]:
import math
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForMaskedLM
from datasets import Dataset

# Load pre-trained model and tokenizer
baseline_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the test data
test_texts = combined_text  # Your test dataset
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"]})

def compute_metrics(eval_pred):
    """Computes perplexity."""
    logits, labels = eval_pred
    # Shift the labels to the left by one and replace the last token with -100
    # to ignore padding tokens during loss calculation.
    shifted_labels = np.roll(labels, -1, axis=-1)
    shifted_labels[:, -1] = -100

    # Calculate loss
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(logits.view(-1, logits.size(-1)), shifted_labels.view(-1))

    # Compute perplexity
    perplexity = math.exp(loss.item())
    return {"perplexity": perplexity, "eval_loss": loss.item()}

# Define Trainer for evaluation
training_args = TrainingArguments(
    output_dir="./baseline_eval",
    per_device_eval_batch_size=16,
)

baseline_trainer = Trainer(
    model=baseline_model,
    args=training_args,
    compute_metrics=compute_metrics
)

# Evaluate the baseline model
results = baseline_trainer.evaluate(eval_dataset=test_dataset)

# Compute perplexity
baseline_perplexity = math.exp(results["eval_loss"])
print(f"Baseline Perplexity: {baseline_perplexity}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: 'eval_loss'